In [1]:
from pts import Trainer
import torch
from gluon.estimator import TradeBotEstimator
from gluon.dataset import generate_backtesting_datasets
from gluon.metrics import compute_validation_metrics
from gluon.plots import plot_four_forecasts
from gluonts.evaluation.backtest import make_evaluation_predictions

import warnings
warnings.filterwarnings("ignore")

/Users/nhassen/opt/anaconda3/envs/CWP-assignment/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/nhassen/opt/anaconda3/envs/CWP-assignment/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
history_factor = 3
backtest_id = 2

metadata, train_data, test_data = generate_backtesting_datasets("electricity_hourly", backtest_id, history_factor, use_cached=False)

Download electricity_hourly_dataset.zip:: 11.3MB [00:21, 551kB/s]                            
/Users/nhassen/opt/anaconda3/envs/CWP-assignment/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/nhassen/opt/anaconda3/envs/CWP-assignment/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/nhassen/opt/anaconda3/envs/CWP-assignment/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/nhassen/opt/anaconda3/envs/CWP-assignment/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-modul

In [3]:
for entry in list(train_data):
    entry["target"] = entry["target"][:20, :]
for entry in list(test_data):
    entry["target"] = entry["target"][:20, :]

In [ ]:
estimator = TradeBotEstimator(
    model_parameters= {
        "gamma":0.8,
        "l_norm": 2,
        "data_normalization":"standardization",
        "loss_normalization":"series",
        "series_embedding_dim":13,
        "input_encoder_layers":3,
        "input_encoding_normalization":True,
        "encoder": {
            "attention_layers":3,
            "attention_heads": 3,
            "attention_dim": 4,
            "attention_feedforward_dim": 12,
        },
        "quantile_decoder":{
             "min_u": 0.01,
             "max_u": 0.99,
            "attentional_quantile": {
                "attention_heads": 3,
                "attention_layers": 3,
                "attention_dim": 12,
                "mlp_layers": 3,
                "mlp_dim": 16,
                "resolution": 50,
            },
        }
    },
    
    num_series = list(train_data)[0]["target"].shape[0],
    history_length = history_factor * metadata.prediction_length,
    prediction_length = metadata.prediction_length,
    freq = metadata.freq,
    trainer = Trainer(
        epochs = 30,
        batch_size = 25,
        num_batches_per_epoch = 25,
        learning_rate = 1e-3,
        weight_decay = 1e-4,
        maximum_learning_rate = 1e-3,
        clip_gradient = 1e3,
        device = torch.device("cpu"),
    ),
    cdf_normalization = False,
    num_parallel_samples = 100,
)

In [6]:
predictor = estimator.train(train_data, test_data)

  0%|          | 0/511 [00:00<?, ?it/s]